In [ ]:
# 명령줄 인수 형태의 리스트 생성
args = {
    "seed": 10,
    "env": "hopper-medium-v2",
    "K": 5,
    "embed_dim": 512,
    "n_layer": 4,
    "n_head": 4,
    "activation_function": "relu",
    "dropout": 0.1,
    "eval_context_length": 5,
    "ordering": 0,
    "eval_rtg": 3600,
    "num_eval_episodes": 10,
    "init_temperature": 0.1,
    "batch_size": 256,
    "learning_rate": 1e-4,
    "weight_decay": 5e-4,
    "warmup_steps": 10000,
    "max_pretrain_iters": 1,
    "num_updates_per_pretrain_iter": 5000,
    "max_online_iters": 1500,
    "online_rtg": 7200,
    "num_online_rollouts": 1,
    "replay_size": 1000,
    "num_updates_per_online_iter": 300,
    "eval_interval": 10,
    "device": "cuda",
    "log_to_tb": True,
    "save_dir": "./exp",
    "exp_name": "default",
    "tags": "model:01"
}


# 생성된 args 리스트 확인
print(args)

from main import Experiment
import utils
import warnings
warnings.filterwarnings('ignore')
utils.set_seed_everywhere(args['seed'])
experiment = Experiment(args)
experiment()



In [2]:
import gym
gym.

<module 'gym.envs' from '/env/lib/python3.8/site-packages/gym/envs/__init__.py'>

In [10]:
import gym
import d4rl
env_name = "antmaze-umaze-v2"
env = gym.make(env_name)
env.spec


Target Goal:  (0.8863395339498391, 8.321076840368889)


/env/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


EnvSpec(entry_point='d4rl.locomotion.ant:make_ant_maze_env', reward_threshold=None, nondeterministic=False, max_episode_steps=700, order_enforce=True, kwargs={'maze_map': [[1, 1, 1, 1, 1], [1, 'r', 0, 0, 1], [1, 1, 1, 0, 1], [1, 'g', 0, 0, 1], [1, 1, 1, 1, 1]], 'reward_type': 'sparse', 'dataset_url': 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_v2/Ant_maze_u-maze_noisy_multistart_False_multigoal_False_sparse_fixed.hdf5', 'non_zero_reset': False, 'eval': True, 'maze_size_scaling': 4.0, 'ref_min_score': 0.0, 'ref_max_score': 1.0, 'v2_resets': True}, namespace=None, name='antmaze-umaze', version=2)

In [64]:
import pickle
import numpy as np
env_name = "antmaze-umaze-v2"
dataset_path = f"./data/{env_name}.pkl"
with open(dataset_path, "rb") as f:
    trajectories = pickle.load(f)
len(trajectories)
print(trajectories[0].keys())
trajectories[0]['observations'].shape

dict_keys(['observations', 'actions', 'rewards', 'next_observations', 'terminals'])


(700, 29)

In [14]:
import pickle
import numpy as np
env_name = "antmaze-umaze-v2"
dataset_path = f"./data/{env_name}.pkl"
with open(dataset_path, "rb") as f:
    trajectories = pickle.load(f)

print(trajectories[0].keys())
print(trajectories[1]['observations'].shape)
states, traj_lens, returns = [], [], []
for path in trajectories:
    states.append(path["observations"])
    traj_lens.append(len(path["observations"]))
    returns.append(path["rewards"].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
num_timesteps = sum(traj_lens)

print("=" * 50)
print(f"Starting new experiment: {env_name}")
print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
print("=" * 50)

sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]
trajectories = [trajectories[ii] for ii in sorted_inds]


dict_keys(['observations', 'actions', 'rewards', 'next_observations', 'terminals'])
(700, 29)
Starting new experiment: antmaze-umaze-v2
10131 trajectories, 998573 timesteps found
Average return: 0.86, std: 0.35
Max return: 1.00, min: 0.00
Average length: 98.57, std: 218.16
Max length: 700.00, min: 1.00


In [8]:
import minari
data_list = minari.list_remote_datasets()
for data in data_list:
    if 'point' in data:
        print(data)

pointmaze-large-dense-v0
pointmaze-large-dense-v1
pointmaze-large-v0
pointmaze-large-v1
pointmaze-medium-dense-v0
pointmaze-medium-dense-v1
pointmaze-medium-v0
pointmaze-medium-v1
pointmaze-open-dense-v0
pointmaze-open-dense-v1
pointmaze-open-v0
pointmaze-open-v1
pointmaze-umaze-dense-v0
pointmaze-umaze-dense-v1
pointmaze-umaze-v0
pointmaze-umaze-v1


In [35]:
import gymnasium as gym
env = gym.make('PointMaze_UMaze-v3')
env.observation_space['observation'].shape


(4,)

In [110]:
import pickle
import numpy as np
import collections
import minari

env_name = "pointmaze-umaze-v1"
actual_env_name = 'PointMaze_Umaze-v3'
dataset = minari.load_dataset("pointmaze-umaze-v1")
dataset = dataset._data.get_episodes([1,2,3,4,5,6,7,8])


In [140]:
import minari
import pickle
import numpy as np
import collections
import minari

env_name = "pointmaze-umaze-v1"
actual_env_name = 'PointMaze_Umaze-v3'
dataset = minari.load_dataset("pointmaze-umaze-v1")
dataset = dataset._data.get_episodes([1,2,3,4,5,6,7,8])

def _load_dataset_minari(env_name, dataset = None): # jensk

    trajectories = dataset
    states, traj_lens, returns = [], [], []
    if 'pointmaze' in env_name :
        # re-label observation. (achieved_goal, desired_goal) -> observation
        print("re-label observation. (achieved_goal, desired_goal) -> observation")
        for path in trajectories :
            achieved_goal = path['observations']['achieved_goal'][1:]
            desired_goal = path['observations']['desired_goal'][1:]
            observation = np.concatenate([achieved_goal, desired_goal], axis=1)
            path['observations'] = observation

    for path in trajectories:
        states.append(path["observations"])
        traj_lens.append(len(path["observations"]))
        returns.append(path["rewards"].sum())
        # for pointmaze
    traj_lens, returns = np.array(traj_lens), np.array(returns)
    states = np.concatenate(states, axis=0)
    state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
    num_timesteps = sum(traj_lens)

    print("=" * 50)
    print(f"Starting new experiment: {env_name}")
    print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
    print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
    print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
    print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
    print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
    print("=" * 50)

    sorted_inds = np.argsort(returns)  # lowest to highest
    num_trajectories = 1
    timesteps = traj_lens[sorted_inds[-1]]
    ind = len(trajectories) - 2
    while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
        timesteps += traj_lens[sorted_inds[ind]]
        num_trajectories += 1
        ind -= 1
    sorted_inds = sorted_inds[-num_trajectories:]

    trajectories = [trajectories[ii] for ii in sorted_inds]
    return trajectories, state_mean, state_std

def _load_dataset_01(env_name) :

    offline_trajs, state_mean, state_std = _load_dataset_minari(env_name, dataset)
    # create ['K'] length trajectories with initial state and final state
    offline_trajs_01 = []
    for traj in offline_trajs:
        traj_len = len(traj["observations"])
        # divide the trajectory into K length. get index of K length
        if traj_len < variant["K"] :
            continue 
        index = np.linspace(0, traj_len - 1, variant["K"]).astype(int)
        #print(f"traj len: {traj_len}, index : {index}")
        # get the state and action of the index
        # get the initial action and final action
        for key in traj.keys() :
            if key in ['id', 'total_timesteps', 'seed' ] :
                continue
            traj[key] = traj[key][index]
        offline_trajs_01.append(traj)
    print(f"offline_trajs_01 : {len(offline_trajs_01)}")
    return offline_trajs_01, state_mean, state_std
    
variant = {}
variant['K'] = 20

offline_trajs_01, state_mean, state_std = _load_dataset_01('pointmaze-umaze-v1')


re-label observation. (achieved_goal, desired_goal) -> observation
Starting new experiment: pointmaze-umaze-v1
8 trajectories, 719 timesteps found
Average return: 1.00, std: 0.00
Max return: 1.00, min: 1.00
Average length: 89.88, std: 39.23
Max length: 145.00, min: 34.00
offline_trajs_01 : 7


In [144]:
idx = 5
print(offline_trajs_01[idx]['observations'])
print(offline_trajs_01[idx]['rewards'])

[[-0.46165204 -1.03910502  0.04136232  0.84216899]
 [-0.72296072 -1.01417648  0.04136232  0.84216899]
 [-0.87453788 -0.99728816  0.04136232  0.84216899]
 [-0.88071978 -0.98238827  0.04136232  0.84216899]
 [-0.74589048 -0.98033173  0.04136232  0.84216899]
 [-0.50590948 -1.00032714  0.04136232  0.84216899]
 [-0.1093681  -1.00844271  0.04136232  0.84216899]
 [ 0.30873235 -1.00538367  0.04136232  0.84216899]
 [ 0.65516719 -1.00288631  0.04136232  0.84216899]
 [ 0.93071127 -0.99625017  0.04136232  0.84216899]
 [ 1.04820873 -0.91352137  0.04136232  0.84216899]
 [ 1.03834278 -0.67888053  0.04136232  0.84216899]
 [ 0.96774437 -0.40819929  0.04136232  0.84216899]
 [ 0.91680077 -0.20358511  0.04136232  0.84216899]
 [ 0.95473953  0.0801339   0.04136232  0.84216899]
 [ 0.97546725  0.43127961  0.04136232  0.84216899]
 [ 0.97459954  0.77961218  0.04136232  0.84216899]
 [ 0.93680558  0.95970287  0.04136232  0.84216899]
 [ 0.75645927  1.02010435  0.04136232  0.84216899]
 [ 0.43379307  0.94768766  0.04

In [125]:
print(traj['observations'])

[[ 0.96220799 -0.72791512  0.99113667 -3.84653731]
 [ 0.99202173 -0.86321949  0.59199016 -3.01313329]
 [ 1.00604544 -0.95926918  0.36015488 -2.03527346]
 [ 1.01928782 -1.01643681  0.15567085 -1.06669603]
 [ 1.0255721  -1.03378042  0.17796668  0.13110652]
 [ 1.02395197 -1.00480758 -0.0417259   1.07911942]
 [ 1.01642863 -0.93813947 -0.19855328  2.01813324]
 [ 1.0178651  -0.83413391  0.05767253  2.9482334 ]
 [ 1.00426739 -0.65216081 -0.22573773  4.0984526 ]
 [ 0.9937511  -0.46679686 -0.06470333  4.92613822]
 [ 0.98991051 -0.28917769 -0.1992853   4.34646464]
 [ 0.99745824 -0.14011154  0.28721259  3.35594806]
 [ 0.99889524  0.01941374 -0.31062212  3.55659662]
 [ 0.97588387  0.18459229 -0.70491504  4.4720928 ]
 [ 0.94739361  0.38368576 -0.67227573  5.13015673]
 [ 0.91866121  0.5692021  -0.56848252  4.29721929]
 [ 0.89686518  0.74692143 -0.15077295  3.06112986]
 [ 0.88467847  0.84487958 -0.61280256  2.08281441]
 [ 0.83655224  0.90535529 -1.55624292  1.25555896]
 [ 0.72368296  0.93935991 -2.72

In [100]:
print(len(trajectories[3]['observations']['desired_goal']))
print(len(trajectories[3]['observations']['achieved_goal']))

25
25


In [84]:
trajectories = dataset.copy()
states, traj_lens, returns = [], [], []
for path in trajectories:
    states.append(path["observations"]['observation'])
    traj_lens.append(len(path["observations"]['observation']))
    returns.append(path["rewards"].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)

states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
num_timesteps = sum(traj_lens)

print("=" * 50)
print(f"Starting new experiment: {env_name}")
print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
print("=" * 50)

sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]
trajectories = [trajectories[ii] for ii in sorted_inds]

print(trajectories[0].keys())
offline_trajs = trajectories
offline_trajs_01 = []
for traj in offline_trajs:
    traj_len = len(traj["observations"]['observation'])
    print(traj_len)
    # divide the trajectory into K length. get index of K length
    if traj_len < 20 :
        continue 
    index = np.linspace(0, traj_len - 1, 20).astype(int)
    #print(f"traj len: {traj_len}, index : {index}")
    # get the state and action of the index
    # get the initial action and final action
    for key in traj.keys() :
        if key in ['id', 'total_timesteps', 'seed', 'actions'] :
            continue
        if hasattr(traj[key], '__len__') :
            print(key)
            print(len(traj[key]))
        if isinstance(traj[key], dict) and 'observation' in traj[key].keys() :
            print(len(traj[key]['observation']))

            traj[key] = traj[key]['observation']
            traj[key] = traj[key][index]
        else :
            traj[key] = traj[key][index]
    offline_trajs_01.append(traj)
print(f"offline_trajs_01 : {len(offline_trajs_01)}")


Starting new experiment: pointmaze-umaze-v1
13289 trajectories, 1013285 timesteps found
Average return: 1.00, std: 0.02
Max return: 1.00, min: 0.00
Average length: 76.25, std: 45.52
Max length: 194.00, min: 1.00
dict_keys(['id', 'total_timesteps', 'seed', 'observations', 'actions', 'rewards', 'terminations', 'truncations'])
17
82
observations
3
82
rewards
81


IndexError: index 81 is out of bounds for axis 0 with size 81

In [78]:
for key in traj.keys() :
    if hasattr(traj[key], '__len__') :
        print(key)
        print(len(traj[key]))

observations
20
actions
81
rewards
81
terminations
81
truncations
81


In [38]:
print(state_mean, state_std)

[ 0.40896365 -0.03382085 -0.02304759  0.00219767] [0.68805839 0.87940597 2.71266236 2.42000897]


In [58]:
# 첫 번째 에피소드 확인
first_episode = dataset[0]

# 첫 번째 에피소드의 타입과 내용 출력
print("Type of first episode:", type(first_episode))
print("Content of first episode:", first_episode)


Type of first episode: <class 'dict'>
Content of first episode: {'id': 0, 'total_timesteps': 162, 'seed': 123, 'observations': {'achieved_goal': array([[-1.15781409e+00, -1.16204705e+00],
       [-1.15543246e+00, -1.15966541e+00],
       [-1.15067485e+00, -1.15490781e+00],
       [-1.14354694e+00, -1.14777989e+00],
       [-1.13405437e+00, -1.13828732e+00],
       [-1.12220276e+00, -1.12747426e+00],
       [-1.10799774e+00, -1.11501784e+00],
       [-1.09144492e+00, -1.10171071e+00],
       [-1.07254988e+00, -1.09077366e+00],
       [-1.05131820e+00, -1.08163646e+00],
       [-1.02775545e+00, -1.07245551e+00],
       [-1.00186718e+00, -1.06344957e+00],
       [-9.73658931e-01, -1.05569180e+00],
       [-9.43136223e-01, -1.05033414e+00],
       [-9.10304570e-01, -1.04515848e+00],
       [-8.75169472e-01, -1.03781344e+00],
       [-8.37736415e-01, -1.03078614e+00],
       [-7.98010872e-01, -1.02615721e+00],
       [-7.55998302e-01, -1.01976628e+00],
       [-7.11704152e-01, -1.01296846e+

In [ ]:

N = dataset["rewards"].shape[0]
data_ = collections.defaultdict(list)

use_timeouts = "timeouts" in dataset

episode_step = 0
paths = []
for i in range(N):
    done_bool = bool(dataset["terminals"][i])
    final_timestep = dataset["timeouts"][i] if use_timeouts else episode_step == 1000 - 1
    for k in ["observations", "next_observations", "actions", "rewards", "terminals"]:
        data_[k].append(dataset[k][i])
    if done_bool or final_timestep:
        episode_step = 0
        episode_data = {k: np.array(data_[k]) for k in data_}
        paths.append(episode_data)
        data_ = collections.defaultdict(list)
    episode_step += 1

returns = np.array([np.sum(p["rewards"]) for p in paths])
num_samples = np.sum([p["rewards"].shape[0] for p in paths])
print(f"Number of samples collected: {num_samples}")
print(f"Trajectory returns: mean = {np.mean(returns)}, std = {np.std(returns)}, max = {np.max(returns)}, min = {np.min(returns)}")

with open(f"{env_name}.pkl", "wb") as f:
    pickle.dump(paths, f)

In [51]:
states, traj_lens, returns = [], [], []
for path in trajectories:
    states.append(path["observations"])
    traj_lens.append(len(path["observations"]))
    returns.append(path["rewards"].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
num_timesteps = sum(traj_lens)

print("=" * 50)
print(f"Starting new experiment: {env_name}")
print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
print("=" * 50)

sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]
trajectories = [trajectories[ii] for ii in sorted_inds]

ValueError: zero-dimensional arrays cannot be concatenated

In [39]:
trajectories[0]['actions']

array([[ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  0.56393635],
       [ 1.        ,  0.70082456],
       [ 1.        ,  0.3696543 ],
       [ 1.        , -0.9818415 ],
       [ 1.        , -0.74478257],
       [ 1.        ,  0.02750837],
       [ 1.        , -0.06430212],
       [ 1.        , -0.51507354],
       [ 1.        , -1.        ],
       [ 1.        , -0.07106009],
       [ 1.        ,  0.91605455],
       [ 1.        , -0.12606966],
       [ 1.        , -1.        ],
       [ 1.        ,  0.7444585 ],
       [ 1.        ,  0.17723933],
       [ 1.        , -0.5203563 ],
       [ 1.        , -0.67444026],
       [ 0.6306356 ,  0.5627092 ],
       [ 0.3944846 , -0.9395471 ],
       [ 0.16204122, -0.01691076],
       [-0.22663297, -1.        ],
       [-0.5117092 , -0.32774252],
       [-1.        ,  0.34729034],
       [-1.        ,  0.5982919 ],
       [-1.        ,

In [46]:
import collections

dataset = {}
dataset['observations'] = []
dataset['actions'] = []
dataset['rewards'] = []
dataset['terminals'] = []
dataset['agent_infos'] = []
dataset['env_infos'] = []
for episodes in trajectories :
    dataset['observations'].append(episodes['observations'])
    dataset['actions'].append(episodes['actions'])
    dataset['rewards'].append(episodes['rewards'])
    dataset['terminals'].append(episodes['terminations'])

N = len(dataset["rewards"])
print(f"Number of trajectories: {N}")

for k in dataset.keys():
    print(dataset[k][0])
    dataset[k] = np.array(dataset[k]).reshape(-1, len(dataset[k][0][0]))


data_ = collections.defaultdict(list)

use_timeouts = False
if "timeouts" in dataset:
    use_timeouts = True

episode_step = 0
paths = []
for i in range(N):
    done_bool = bool(dataset["terminals"][i])
    if use_timeouts:
        final_timestep = dataset["timeouts"][i]
    else:
        final_timestep = episode_step == 1000 - 1
    for k in [
        "observations",
        "next_observations",
        "actions",
        "rewards",
        "terminals",
    ]:
        data_[k].append(dataset[k][i])
    if done_bool or final_timestep:
        episode_step = 0
        episode_data = {}
        for k in data_:
            episode_data[k] = np.array(data_[k])
        paths.append(episode_data)
        data_ = collections.defaultdict(list)
    episode_step += 1

returns = np.array([np.sum(p["rewards"]) for p in paths])
num_samples = np.sum([p["rewards"].shape[0] for p in paths])
print(f"Number of samples collected: {num_samples}")
print(
    f"Trajectory returns: mean = {np.mean(returns)}, std = {np.std(returns)}, max = {np.max(returns)}, min = {np.min(returns)}"
)

with open(f"{name}.pkl", "wb") as f:
    pickle.dump(paths, f)


Number of trajectories: 13289
{'achieved_goal': array([[-1.15781409e+00, -1.16204705e+00],
       [-1.15543246e+00, -1.15966541e+00],
       [-1.15067485e+00, -1.15490781e+00],
       [-1.14354694e+00, -1.14777989e+00],
       [-1.13405437e+00, -1.13828732e+00],
       [-1.12220276e+00, -1.12747426e+00],
       [-1.10799774e+00, -1.11501784e+00],
       [-1.09144492e+00, -1.10171071e+00],
       [-1.07254988e+00, -1.09077366e+00],
       [-1.05131820e+00, -1.08163646e+00],
       [-1.02775545e+00, -1.07245551e+00],
       [-1.00186718e+00, -1.06344957e+00],
       [-9.73658931e-01, -1.05569180e+00],
       [-9.43136223e-01, -1.05033414e+00],
       [-9.10304570e-01, -1.04515848e+00],
       [-8.75169472e-01, -1.03781344e+00],
       [-8.37736415e-01, -1.03078614e+00],
       [-7.98010872e-01, -1.02615721e+00],
       [-7.55998302e-01, -1.01976628e+00],
       [-7.11704152e-01, -1.01296846e+00],
       [-6.65133856e-01, -1.00742612e+00],
       [-6.16292836e-01, -1.00350325e+00],
      

KeyError: 0

In [7]:
states, traj_lens, returns = [], [], []
for path in trajectories:
    states.append(path["observations"])
    traj_lens.append(len(path["observations"]))
    returns.append(path["rewards"].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)

# used for input normalization
print(len(states[5]))
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
num_timesteps = sum(traj_lens)

print("=" * 50)
print(f"Starting new experiment: {env_name}")
print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
print("=" * 50)

sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]
trajectories = [trajectories[ii] for ii in sorted_inds]


3


ValueError: zero-dimensional arrays cannot be concatenated

In [46]:
#for trajectory in all_data:
    #print(trajectory.keys())
    #dict_keys(['id', 'total_timesteps', 'seed', 'observations', 'actions', 'rewards', 'terminations', 'truncations'])
import numpy as np
# concatenate all trajectories into one big trajectory
states, actions, rewards, terminations, truncations = [], [], [], [], []
for trajectory in all_data:
    states.append(trajectory["observations"])
    actions.append(trajectory["actions"])
    rewards.append(trajectory["rewards"])
    terminations.append(trajectory["terminations"])
    truncations.append(trajectory["truncations"])
# concatenate along the leading axis


print(len(states))
print(states[0].keys())

13289
dict_keys(['achieved_goal', 'desired_goal', 'observation'])


In [ ]:
import minari

datasets = []
minari_datasets_name = 'pointmaze-umaze-v1'
dataset = minari.load_dataset("pointmaze-umaze-v1")
env_name = 'PointMaze_Umaze-v3'
#dataset.episode_indices
all_data = dataset._data.get_episodes(dataset.episode_indices)
all_data[0].keys()


In [4]:
"""
Copyright (c) Meta Platforms, Inc. and affiliates.

This source code is licensed under the CC BY-NC license found in the
LICENSE.md file in the root directory of this source tree.
"""

import gymnasium as gym
import numpy as np
import collections
import pickle
import d4rl
import minari

datasets = []
minari_datasets_name = 'pointmaze-umaze-v1'

dataset = minari.load_dataset("pointmaze-umaze-v1")
#dataset.episode_indices
all_data = dataset._data.get_episodes(dataset.episode_indices)
all_data[0].keys()




/env/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_dataset to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_dataset` for environment variables or `env.get_wrapper_attr('get_dataset')` that will search the reminding wrappers.
  logger.warn(


AttributeError: 'PointMazeEnv' object has no attribute 'get_dataset'

In [1]:
import gymnasium as gym
gym.pprint_registry()

===== classic_control =====
Acrobot-v1                                                    CartPole-v0                                                   CartPole-v1
MountainCar-v0                                                MountainCarContinuous-v0                                      Pendulum-v1
===== phys2d =====
phys2d/CartPole-v0                                            phys2d/CartPole-v1                                            phys2d/Pendulum-v0
===== box2d =====
BipedalWalker-v3                                              BipedalWalkerHardcore-v3                                      CarRacing-v2
LunarLander-v2                                                LunarLanderContinuous-v2
===== toy_text =====
Blackjack-v1                                                  CliffWalking-v0                                               FrozenLake-v1
FrozenLake8x8-v1                                              Taxi-v3
===== tabular =====
tabular/Blackjack-v0                           